# Homework 4

## FINM 37500: Fixed Income Derivatives

* Matheus Raka Pradnyatama
* matheusraka@uchicago.edu

#### Winter 2025

In [1]:
import pandas as pd
import numpy as np
import datetime
import holidays
import seaborn as sns
import math

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.linear_model import LinearRegression

from scipy.optimize import minimize
from scipy import interpolate
from scipy.optimize import fsolve
from scipy.stats import norm

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import numpy.polynomial.polynomial as poly
# The code here is written with the help of OpenAI's ChatGPT

***

### Data

The file `data/ratetree_data_2025-01-31.xlsx` has a binomial tree of interest rates fit to...
* discount curves from `cap_curves_2025-01-31.xlsx`
* implied vols from `cap_curves_2025-01-31.xlsx`

Note the following...
* Suppose the present date is `2025-01-31`.
* The rates are continuously compounded.
* The rates are for the following quarter. So teh rate at $t=0$ is the continuously compounded rate for the interval $t=0$ to $t=.25$.

Take this binomial tree as given; there is no need to fit it yourself.

In [2]:
import pandas as pd

DATE = '2025-01-31'
FILEIN = f'../data/ratetree_data_{DATE}.xlsx'
sheet_tree = 'rate tree'

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

ratetree.style.format('{:.1%}',na_rep='').format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,4.2%,4.3%,4.4%,4.8%,5.1%,6.4%,7.6%,9.1%,10.1%,11.8%,13.5%,15.3%,16.9%,19.2%,22.7%,25.9%,28.2%,30.6%,34.7%,40.3%
1,,3.9%,4.0%,4.2%,4.4%,5.2%,6.0%,7.0%,7.8%,9.0%,10.4%,11.8%,13.0%,14.8%,17.4%,19.9%,21.7%,23.7%,27.0%,31.2%
2,,,3.6%,3.7%,3.8%,4.2%,4.7%,5.4%,6.0%,7.0%,8.0%,9.1%,10.1%,11.4%,13.4%,15.3%,16.7%,18.4%,20.9%,24.2%
3,,,,3.3%,3.2%,3.3%,3.7%,4.2%,4.6%,5.3%,6.2%,7.0%,7.8%,8.8%,10.3%,11.8%,12.9%,14.2%,16.3%,18.8%
4,,,,,2.8%,2.7%,2.9%,3.3%,3.6%,4.1%,4.7%,5.4%,6.0%,6.8%,7.9%,9.1%,10.0%,11.0%,12.6%,14.6%
5,,,,,,2.2%,2.3%,2.5%,2.7%,3.2%,3.6%,4.2%,4.6%,5.2%,6.1%,7.0%,7.7%,8.6%,9.8%,11.3%
6,,,,,,,1.8%,2.0%,2.1%,2.4%,2.8%,3.2%,3.6%,4.0%,4.7%,5.4%,5.9%,6.6%,7.6%,8.8%
7,,,,,,,,1.5%,1.6%,1.9%,2.2%,2.5%,2.7%,3.1%,3.6%,4.1%,4.6%,5.1%,5.9%,6.8%
8,,,,,,,,,1.3%,1.4%,1.7%,1.9%,2.1%,2.4%,2.8%,3.2%,3.5%,4.0%,4.6%,5.3%


***

# 1. Binomial Tree Pricing - Bond

### The Bond

Consider a vanilla (non-callable) bond with the following parameters...
* `T=5`
* coupon rate is `4.41%`
* coupons are semiannual

Note that this is essentially the hypothetical bond priced in HW 1.

### 1.1

Create and display a tree of cashflows from the bond, corresponding to each node of the tree (state and time) seen in the interest rate tree.

Note that the cashflows do not depend on the interest rates. Thus, report the cashflows at the time (in the column) they are actually paid out. The final payoff (face plus coupon) occurs at $T$, which is beyond the interest rate tree. You are welcome to add a column for $T$ or to consider this payoff separately and leave it out of the tree.

In [3]:
cpn_rate = 0.0441 # Annual Coupon Rate
cpn_payment = 100 * cpn_rate / 2 # Semi-annual coupon payment

bond_tree = ratetree.copy() # Copy the structure of the interest rate tree

# Set the initial time step (t = 0) to 0 as no cash flow occurs at issuance
bond_tree[0].iloc[0] = 0

# Iterate through the tree to assign cash flows
for col in bond_tree.columns[1:]:
    if round(4 * col) % 2: # Check if it's NOT a coupon payment time
        for j in range(20): # Loop through each state (20 states at the final step)
            
            # If the time does not correspond to a semiannual coupon date, we set cash flow to 0.
            if not np.isnan(bond_tree[col].iloc[j]): 
                bond_tree[col].iloc[j] = 0
                
    else: # Assign coupon payments at semi-annual intervals
        for j in range(20): 
            # If the time does correspond to a semiannual coupon date, we assign cpn_payment
            if not np.isnan(bond_tree[col].iloc[j]):
                bond_tree[col].iloc[j] = cpn_payment

bond_tree.style.format('{:.3f}',na_rep='').format_index('{:.2f}',axis=1)

/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/3669109459.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  bond_tree[0].iloc[0] = 0
/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/3669109459.py:16: Futur

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0.000,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
1,,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
2,,,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
3,,,,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
4,,,,,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
5,,,,,,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
6,,,,,,,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
7,,,,,,,,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000
8,,,,,,,,,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000,2.205,0.000


In [4]:
print(f"Final payment is ${100 + cpn_payment}")

Final payment is $102.205


### 1.2.

Create and display a tree of values of the bond. Do this for the quotes as
* clean quotes
* dirty quotes

Given the semiannual coupons and quarterly tree steps, the clean and dirty will coincide at $t=0, .5, 1,...$.

Do the valuation by...
* setting the value at $T$ as the face plus final coupon.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

In [ ]:
# Binomial Tree - Dirty Quotes

# bond_tree contains the bond cash flows at each node.
# binomial_tree will be used to store the computed bond values at each node.
binomial_tree = bond_tree.copy()

for j in range(20):
    # Compute the Final Payoff at Maturity (T = 4.75)
    # Discount the final payment of 100 + Coupon payment
    # Discounted Value = Cash Flow / e^(rate * delta_t)
    # Interest rate at T=4.75
    # Delta_t is 0.25 (the tree uses quarterly steps)
    binomial_tree[4.75].iloc[j] = (100 + cpn_payment) / np.exp(ratetree[4.75].iloc[j] * 0.25)
    
# Iterate backwards from the second-to-last time step (4.50) to time 0
for i, col in enumerate(binomial_tree.columns[18:0:-1]):
    # Since the binomial tree recombines, the number of states at each time step is one less than the next step.
    for j in range(19-i):
        # Compute the expected future bond price by averaging:
        # The upper node price at time col + 0.25 (jth state).
        # The lower node price at time col + 0.25 (j+1th state).
        # Since each up/down movement is equally likely, we take the simple average.
        avg_price = (binomial_tree[col + 0.25].iloc[j] + 
                     binomial_tree[col + 0.25].iloc[j+1]) / 2
        
        # Discounted Price = Price / e^(rate * delta_t)
        binomial_tree[col].iloc[j] = avg_price / np.exp(ratetree[col].iloc[j] * 0.25) + binomial_tree[col].iloc[j]

# Binomial Estimation for Dirty Price

# average price = (price_up_scenario + price_down_scenario)/2
avg_price_dirty = (binomial_tree[0.25].iloc[0] + binomial_tree[0.25].iloc[1]) / 2

# Discount the average price by rate at time 0 and delta_t = 0.25 (we are discounting from t=0.25 to t=0)
binom_dirty_price = avg_price_dirty / np.exp(ratetree[0].iloc[0] * 0.25)

# Put it in the binonmial tree
binomial_tree[0].iloc[0] = binom_dirty_price

binomial_tree.style.format('{:.1f}',na_rep='').format_index('{:.2f}',axis=1)

/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/1948914543.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  binomial_tree[4.75].iloc[j] = (100 + cpn_payment) / np.exp(ratetree[4.75].iloc[j] * 0.25)
/var/folders/pk/dv8l_x

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,101.9,101.0,100.0,96.7,95.4,91.8,90.6,87.2,86.3,83.2,82.7,80.2,80.3,78.4,79.4,78.9,81.6,83.0,87.9,92.4
1,,104.9,104.2,101.2,100.3,97.0,96.0,92.9,92.1,89.2,88.8,86.3,86.4,84.5,85.2,84.4,86.7,87.4,91.4,94.5
2,,,107.6,104.9,104.2,101.3,100.5,97.6,97.0,94.2,93.8,91.4,91.4,89.4,90.1,89.0,90.8,90.9,94.1,96.2
3,,,,107.9,107.5,104.7,104.1,101.4,100.9,98.2,97.9,95.5,95.5,93.5,94.0,92.7,94.1,93.8,96.3,97.5
4,,,,,110.0,107.5,107.0,104.4,104.0,101.4,101.2,98.8,98.8,96.8,97.2,95.7,96.8,96.1,98.1,98.6
5,,,,,,109.7,109.4,106.8,106.5,104.0,103.8,101.5,101.5,99.4,99.7,98.1,98.9,97.9,99.5,99.4
6,,,,,,,111.2,108.7,108.5,106.0,105.9,103.6,103.6,101.5,101.7,99.9,100.6,99.3,100.6,100.0
7,,,,,,,,110.2,110.0,107.6,107.5,105.2,105.2,103.1,103.3,101.4,101.9,100.4,101.4,100.5
8,,,,,,,,,111.2,108.9,108.8,106.5,106.5,104.4,104.5,102.6,103.0,101.3,102.1,100.9


In [6]:
# 1.2 clean
# note that dirty prices on coupon dates DON'T align
# because dirty prices are quoted right before coupon payment

clean_binomial_tree = binomial_tree.copy()

# Loop over each time step
# i = index (0 for t=0, 1 for t=0.25, ...)
# col = actual time value (0, 0.25, 0.50, ...)
for i, col in enumerate(clean_binomial_tree.columns):
    
    # For every midpoint (3 months after the last coupon)
    # t = 0.25, 0.75, 1.25, ...
    # Only half of the next coupon has accrued
    # Clean Price = Dirty Price - (Coupon Payment / 2)
    if i % 2:
        clean_binomial_tree[col] -= cpn_payment / 2
        
    # On other dates, the full previous coupon has accrued
    # Clean Price = Dirty Price - Coupon Payment
    elif i:
        clean_binomial_tree[col] -= cpn_payment

# At t=0, the clean price is the same as the dirty price
clean_binomial_tree[0].iloc[0] = binom_dirty_price

clean_binomial_tree.style.format('{:.1f}',na_rep='').format_index('{:.2f}',axis=1)

/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/2150597750.py:25: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  clean_binomial_tree[0].iloc[0] = binom_dirty_price


time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,101.9,99.9,97.8,95.6,93.2,90.7,88.4,86.1,84.1,82.1,80.5,79.1,78.1,77.3,77.2,77.8,79.4,81.9,85.7,91.3
1,,103.8,102.0,100.1,98.1,95.9,93.8,91.8,89.9,88.1,86.6,85.2,84.2,83.4,83.0,83.3,84.5,86.3,89.1,93.4
2,,,105.4,103.8,102.0,100.2,98.3,96.5,94.8,93.1,91.6,90.3,89.2,88.3,87.9,87.9,88.6,89.8,91.9,95.1
3,,,,106.8,105.2,103.6,101.9,100.3,98.7,97.1,95.7,94.4,93.3,92.4,91.8,91.6,91.9,92.7,94.1,96.4
4,,,,,107.8,106.4,104.8,103.3,101.8,100.3,99.0,97.7,96.6,95.7,95.0,94.6,94.6,95.0,95.9,97.4
5,,,,,,108.6,107.1,105.7,104.3,102.9,101.6,100.4,99.3,98.3,97.5,97.0,96.7,96.8,97.3,98.3
6,,,,,,,109.0,107.6,106.3,104.9,103.7,102.5,101.4,100.4,99.5,98.8,98.4,98.2,98.3,98.9
7,,,,,,,,109.1,107.8,106.5,105.3,104.1,103.0,102.0,101.1,100.3,99.7,99.3,99.2,99.4
8,,,,,,,,,109.0,107.8,106.6,105.4,104.3,103.3,102.3,101.5,100.8,100.2,99.9,99.8


### 1.3.

The binomial-estimated price of the bond is the initial node of the value tree.

Report this along with the price of the bond you would get from the usual simple formula for such a bond. 
* Consider pricing it with the $T$ interval swap rate (used similar to a ytm) from the file `cap_curves_2025-01-31.xlsx`.
* If you do this, recall that the swap rate given in that file is quarterly-compounded, so you would need to convert it to semiannual compounding before plugging it into the usual closed-form ytm-pricing formula.

In [7]:
# Binomial Estimation for Dirty Price

# average price = (price_up_scenario + price_down_scenario)/2
avg_price_dirty = (binomial_tree[0.25].iloc[0] + binomial_tree[0.25].iloc[1]) / 2

# Discount the average price by rate at time 0 and delta_t = 0.25 (we are discounting from t=0.25 to t=0)
binom_dirty_price = avg_price_dirty / np.exp(ratetree[0].iloc[0] * 0.25)

# Put it in the binonmial tree
binomial_tree[0].iloc[0] = binom_dirty_price
clean_binomial_tree[0].iloc[0] = binom_dirty_price

print(f"Initial binomial-estimated dirty price: ${round(binom_dirty_price,6)}")

# At t=0, the dirty price is the same as the clean price
print(f"Initial binomial-estimated clean price: ${round(binom_dirty_price,6)}")

Initial binomial-estimated dirty price: $101.869401
Initial binomial-estimated clean price: $101.869401


/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/2528377482.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  binomial_tree[0].iloc[0] = binom_dirty_price
/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/25

### Note:

An easy check on your code is whether it will correctly price a zero-coupon bond at a price that matches the "discounts" in the `cap_curves` data file.

***

# 2. Pricing the Callable - European

### 2.1.

Calculate and display value tree of a European-style call option on the bond analyzed in part `1`.
* `$T_o = 3$`. That is, the time-to-expiration is 3 years.
* `$K=100$`. That is, the strike is 100. This is a clean strike, meaning exercise requires paying the strike plus any accrued interest.

Do so by 
* setting the value at the time of expiration, using the value of the bond for each node at that time.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

Note that...
* the tree of call values will not be the same size as the tree of bond values. The former goes only to $T_o=3$.

Pricing the European Call Option on the Bond

This code constructs a binomial tree for a European-style call option on the bond by:

Initializing the tree using the clean bond value tree.

Setting terminal values at T_0 = 3

Performing backward induction to compute the call option value at each node.

In [8]:
import numpy as np
import pandas as pd

# Step 1: Initialize the call option tree
call_tree = clean_binomial_tree.iloc[:13].copy()  # Copy only the first 13 states (corresponding to 3 years)
call_tree.drop([3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75], axis=1, inplace=True)  # Remove time steps beyond T_0 = 3

# Step 2: Define strike price (clean price strike)
K = 100  # The exercise price of the call option

# Step 3: Set terminal values (at T_0 = 3)
for j in range(13):  # Iterate over all possible states at T_0
    call_tree[3].iloc[j] = max(call_tree[3].iloc[j] - K, 0)  # Call option value = max(Bond Price - Strike, 0)

# Step 4: Perform backward induction to compute call option values at earlier time steps
for i, col in enumerate(call_tree.columns[11:0:-1]):  # Reverse iterate through time steps from T = 2.5 back to T = 0.25
    for j in range(12 - i):  # Reduce the number of states as we move backward
        avg_price = (call_tree[col + 0.25].iloc[j] + call_tree[col + 0.25].iloc[j + 1]) / 2  # Expected call value
        call_tree[col].iloc[j] = avg_price / np.exp(ratetree[col].iloc[j] * 0.25)  # Discounted expected value

# Step 5: Compute the initial value of the call option at T = 0
avg_price = (call_tree[0.25].iloc[0] + call_tree[0.25].iloc[1]) / 2  # Expected price at T = 0.25
call_tree[0].iloc[0] = avg_price / np.exp(ratetree[0].iloc[0] * 0.25)  # Discount back to T = 0

# Step 6: Display the formatted call option value tree
call_tree.style.format('{:.2f}', na_rep='').format_index('{:.2f}', axis=1)


/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/3248307878.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  call_tree[3].iloc[j] = max(call_tree[3].iloc[j] - K, 0)  # Call option value = max(Bond Price - Strike, 0)
/var/

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,1.65,1.20,0.80,0.48,0.24,0.09,0.02,0.00,0.00,0.00,0.00,0.00,0.00
1,,2.13,1.62,1.15,0.73,0.39,0.16,0.04,0.00,0.00,0.00,0.00,0.00
2,,,2.68,2.14,1.59,1.08,0.63,0.29,0.08,0.00,0.00,0.00,0.00
3,,,,3.28,2.72,2.13,1.55,0.99,0.51,0.17,0.00,0.00,0.00
4,,,,,3.89,3.35,2.76,2.13,1.49,0.87,0.34,0.00,0.00
5,,,,,,4.48,3.98,3.43,2.81,2.14,1.42,0.68,0.00
6,,,,,,,5.02,4.58,4.08,3.52,2.89,2.18,1.37
7,,,,,,,,5.51,5.13,4.69,4.20,3.64,3.02
8,,,,,,,,,5.94,5.60,5.22,4.79,4.31


### 2.2.

Show the value tree of the callable bond by subtracting the call value tree from the (subset $t\le T_o$ of the) bond value tree (calculated in part `1`.) Do this for both
* clean
* dirty

In [9]:
# 2.2 dirty

# Step 1: Initialize the callable bond tree (Dirty Prices)
callable_tree = binomial_tree.iloc[:13].copy()  # Copy only relevant rows up to T_0 = 3
callable_tree.drop([3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75], axis=1, inplace=True)  # Remove time steps beyond T_0 = 3

# Step 2: Compute the callable bond value at each node
for i, col in enumerate(callable_tree.columns):  # Iterate over all time steps (T = 0 to T = 3)
    for j in range(i + 1):  # Iterate over valid states (increasing with time)
        callable_tree.loc[j, col] -= call_tree.loc[j, col]  # Callable bond = Bond value - Call option value

callable_tree.style.format('{:.2f}', na_rep='').format_index('{:.2f}', axis=1)


time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,100.22,99.82,99.23,96.20,95.17,91.71,90.54,87.21,86.28,83.23,82.68,80.18,80.27
1,,102.74,102.57,100.03,99.54,96.61,95.86,92.86,92.13,89.23,88.76,86.30,86.36
2,,,104.93,102.73,102.64,100.19,99.88,97.29,96.88,94.18,93.78,91.36,91.41
3,,,,104.59,104.73,102.58,102.59,100.38,100.36,98.03,97.87,95.49,95.53
4,,,,,106.15,104.13,104.29,102.27,102.52,100.57,100.83,98.81,98.84
5,,,,,,105.19,105.37,103.39,103.70,101.87,102.37,100.79,101.48
6,,,,,,,106.15,104.13,104.40,102.52,102.97,101.38,102.20
7,,,,,,,,104.70,104.90,102.95,103.30,101.57,102.20
8,,,,,,,,,105.29,103.28,103.56,101.72,102.20


In [10]:
# 2.2 clean

clean_callable_tree = clean_binomial_tree.iloc[:13].copy()
clean_callable_tree.drop([3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75], axis=1, inplace=True)

for i, col in enumerate(clean_callable_tree.columns):
    for j in range(i+1):
        clean_callable_tree[col].iloc[j] -= call_tree[col].iloc[j]

clean_callable_tree.style.format('{:.2f}',na_rep='').format_index('{:.2f}',axis=1)

/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/3028831881.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  clean_callable_tree[col].iloc[j] -= call_tree[col].iloc[j]


time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,100.22,98.72,97.03,95.09,92.97,90.61,88.34,86.11,84.07,82.13,80.48,79.08,78.07
1,,101.64,100.37,98.93,97.33,95.51,93.66,91.76,89.93,88.13,86.55,85.19,84.16
2,,,102.72,101.63,100.44,99.09,97.67,96.18,94.67,93.08,91.58,90.26,89.21
3,,,,103.48,102.53,101.48,100.39,99.27,98.16,96.93,95.67,94.39,93.32
4,,,,,103.94,103.03,102.08,101.17,100.32,99.46,98.63,97.71,96.64
5,,,,,,104.09,103.17,102.29,101.49,100.77,100.17,99.68,99.28
6,,,,,,,103.95,103.03,102.19,101.41,100.77,100.28,100.00
7,,,,,,,,103.60,102.70,101.85,101.10,100.47,100.00
8,,,,,,,,,103.09,102.18,101.35,100.61,100.00


### 2.3.

Report the initial node value of the call option and of the callable bond.

In a table, compare these to what you got in HW 1 as the value of the embedded call and the value of the callable bond.
* In `HW 1`, we were valuing from a date nearly two weeks later, `2025-02-13`. This difference in the timing means we wouldn't expect the values to match exactly, even if the methods were entirely consistent.

In [11]:
# 2.3

hw1_embedded_call = 2.90
hw1_callable_price = 98.67

comparison_df = pd.DataFrame({
    'qty': ['embedded call', 'callable price'],
    'hw1': [hw1_embedded_call, hw1_callable_price],
    'hw4': [call_tree[0].iloc[0], callable_tree[0].iloc[0]]
}).set_index('qty')

comparison_df.style.format('{:.2f}',na_rep='')

,hw1,hw4
qty,,
embedded call,2.90,1.65
callable price,98.67,100.22


***

# 3. Pricing the Callable - American

### 3.1.

Re-do part `2.`, but this time, make the option a **American** style. That is, allow it to be exercised at any node.
* Report the tree of callable-bond values.
* How does this compare to the European-style?

#### Note
To do this valuation, go through the procedure in `2.1.`, but at each node, compare the value for the call with the value of the payoff function based on the vanilla bond's value at that node. Take the maximum of the two. If you code this carefully, you can simply add a line of code to what you did in `2.1`.

In [ ]:
import numpy as np
import pandas as pd

# Step 1: Initialize the call option tree (American-style)
am_call_tree = clean_binomial_tree.iloc[:13].copy()  # Copy only the first 13 rows (up to T_0 = 3)
am_call_tree.drop([3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75], axis=1, inplace=True)  # Remove time steps beyond T_0 = 3

# Step 2: Define the strike price
K = 100  # The clean strike price

# Step 3: Set terminal values at expiration (T_0 = 3)
for j in range(13):  # Iterate over all possible states at T_0
    am_call_tree[3].iloc[j] = max(am_call_tree[3].iloc[j] - K, 0)  # Call option payoff at expiration

# Step 4: Perform backward induction to compute call option values at earlier time steps
for i, col in enumerate(am_call_tree.columns[11:0:-1]):  # Reverse iterate through time steps from T = 2.5 back to T = 0.25
    for j in range(12 - i):  # Reduce the number of valid states as we move backward
        # Compute the expected continuation value (risk-neutral expected price)
        avg_price = (am_call_tree[col + 0.25].iloc[j] + am_call_tree[col + 0.25].iloc[j + 1]) / 2
        tent_price = avg_price / np.exp(ratetree[col].iloc[j] * 0.25)  # Discount to present value

        # Compare with immediate exercise value and take the max
        am_call_tree[col].iloc[j] = max(tent_price, clean_binomial_tree[col].iloc[j] - K)

# Step 5: Compute the initial value of the call option at T = 0
avg_price = (am_call_tree[0.25].iloc[0] + am_call_tree[0.25].iloc[1]) / 2  # Expected price at T = 0.25
tent_price = avg_price / np.exp(ratetree[0].iloc[0] * 0.25)  # Discount back to T = 0
am_call_tree[0].iloc[0] = max(tent_price, clean_binomial_tree[0].iloc[0] - K)  # Apply early exercise rule

am_call_tree.style.format('{:.2f}', na_rep='').format_index('{:.2f}', axis=1)

/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/619095520.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  am_call_tree[3].iloc[j] = max(am_call_tree[3].iloc[j] - K, 0)  # Call option payoff at expiration
/var/folders/pk

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,2.96,1.93,1.15,0.61,0.27,0.10,0.02,0.00,0.00,0.00,0.00,0.00,0.00
1,,4.04,2.76,1.72,0.96,0.46,0.17,0.04,0.00,0.00,0.00,0.00,0.00
2,,,5.40,3.85,2.52,1.48,0.76,0.31,0.08,0.00,0.00,0.00,0.00
3,,,,6.76,5.25,3.61,2.24,1.22,0.55,0.17,0.00,0.00,0.00
4,,,,,7.83,6.37,4.84,3.30,1.91,0.95,0.34,0.00,0.00
5,,,,,,8.57,7.15,5.71,4.30,2.90,1.59,0.68,0.00
6,,,,,,,8.97,7.61,6.27,4.94,3.66,2.46,1.37
7,,,,,,,,9.11,7.82,6.53,5.29,4.11,3.02
8,,,,,,,,,9.03,7.79,6.57,5.41,4.31


### 3.2.

In which nodes will the American-style callable bond be exercised?

In [13]:
import numpy as np
import pandas as pd

# Step 1: Initialize the early exercise identification tree (copy of European call tree)
am_info_tree = call_tree.copy()  # Copy the European call value tree

# Step 2: Iterate through the tree and check for early exercise
for i, col in enumerate(am_info_tree.columns):  # Iterate over all time steps
    for j in range(i + 1):  # Iterate over valid states at each time step
        # Check if early exercise is optimal:
        # If (Vanilla Bond Price - Strike) > European Call Value, set node to True (1), else False (0)
        am_info_tree.loc[j, col] = (binomial_tree.loc[j, col] - 100 > am_info_tree.loc[j, col])

am_info_tree.style.format(na_rep='').format_index('{:.2f}', axis=1)


/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/2999270554.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  am_info_tree.loc[j, col] = (binomial_tree.loc[j, col] - 100 > am_info_tree.loc[j, col])
/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/2999270554.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  am_info_tree.loc[j, col] = (binomial_tree.loc[j, col] - 100 > am_info_tree.loc[j, col])
/var/folders/pk/dv8l_x052_77y1vty7qynnrm0000gn/T/ipykernel_46122/2999270554.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. 

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,True,False,False,False,False,False,False,False,False,False,False,False,False
1,,True,True,True,False,False,False,False,False,False,False,False,False
2,,,True,True,True,True,False,False,False,False,False,False,False
3,,,,True,True,True,True,True,True,False,False,False,False
4,,,,,True,True,True,True,True,True,True,False,False
5,,,,,,True,True,True,True,True,True,True,True
6,,,,,,,True,True,True,True,True,True,True
7,,,,,,,,True,True,True,True,True,True
8,,,,,,,,,True,True,True,True,True


True means the American style callable bond will be exercised

False means the American style callable bond will not be exercised.


***

# 4. Pricing the Callable - Bermudan

#### This Section is NOT REQUIRED and NOT EXPECTED
Still, it is not much additional work, and some of you may find it interesting. It also illustrates the power of binomial trees in how easily they handle the Bermudan style. 

### 4.1.

Re-do part `3`, but this time with **Bermudan** style exercise. 
* This corresponds to the Freddie Mac bond in `HW 1`.
* Note that the option value tree will now go all the way to $T$.

As a reminder, the Bermudan style can be exercised as early as $T_o$ all the way to $T$. It can only be exercised on specific dates at 3-month intervals, but in our quarterly-spaced tree, this means every node from $T_o$ onward.

### 4.2.

Compare the valuation to the market quote in `HW 1`.

***